In [7]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model, load_model
import pickle as pk
import numpy as np
import tensorflow as tf
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
train_dir = r"Dataset\Train"
validation_dir = r"Dataset\Validation"
test_dir = r"Dataset\Test"

In [3]:
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [4]:
batch_size = 32
image_size = (256, 256)

In [11]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode="binary"
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode="binary"
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode="binary"
)

Found 140002 images belonging to 2 classes.
Found 39428 images belonging to 2 classes.
Found 10905 images belonging to 2 classes.


In [12]:
def deepfake_classifier(image_size):
    input_img = Input(shape=(image_size[0], image_size[1], 3))
    x = Conv2D(32, (3, 3), activation="relu", padding="same")(input_img)
    x = MaxPooling2D((2, 2), padding="same")(x)
    x = Conv2D(64, (3, 3), activation="relu", padding="same")(x)
    x = MaxPooling2D((2, 2), padding="same")(x)
    x = Conv2D(128, (3, 3), activation="relu", padding="same")(x)
    encoded = MaxPooling2D((2, 2), padding="same")(x)
    flat = Flatten()(encoded)
    dense = Dense(128, activation="relu")(flat)
    output = Dense(1, activation="sigmoid")(
        dense
    )
    classifier = Model(input_img, output)
    classifier.compile(
        optimizer="adam",
        loss="binary_crossentropy",
        metrics=["accuracy", "Precision", "Recall"],
    )
    return classifier

In [13]:
classifier = deepfake_classifier(image_size)
classifier.fit(train_generator, epochs=5, validation_data=validation_generator)

Epoch 1/5


d:\Python\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


4376/4376 ━━━━━━━━━━━━━━━━━━━━ 2156s 492ms/step - Precision: 0.7814 - Recall: 0.8076 - accuracy: 0.7906 - loss: 0.4259 - val_Precision: 0.8695 - val_Recall: 0.9105 - val_accuracy: 0.8865 - val_loss: 0.2648
Epoch 2/5
4376/4376 ━━━━━━━━━━━━━━━━━━━━ 2158s 493ms/step - Precision: 0.9236 - Recall: 0.9398 - accuracy: 0.9310 - loss: 0.1689 - val_Precision: 0.9007 - val_Recall: 0.9170 - val_accuracy: 0.9076 - val_loss: 0.2232
Epoch 3/5
4376/4376 ━━━━━━━━━━━━━━━━━━━━ 2173s 497ms/step - Precision: 0.9509 - Recall: 0.9640 - accuracy: 0.9570 - loss: 0.1068 - val_Precision: 0.9135 - val_Recall: 0.9216 - val_accuracy: 0.9168 - val_loss: 0.2146
Epoch 4/5
4376/4376 ━━━━━━━━━━━━━━━━━━━━ 2155s 492ms/step - Precision: 0.9686 - Recall: 0.9770 - accuracy: 0.9728 - loss: 0.0687 - val_Precision: 0.8994 - val_Recall: 0.9490 - val_accuracy: 0.9211 - val_loss: 0.2426
Epoch 5/5
4376/4376 ━━━━━━━━━━━━━━━━━━━━ 2171s 496ms/step - Precision: 0.9794 - Recall: 0.9857 - accuracy: 0.9825 - loss: 0.0446 - val_Precision: 

In [31]:
precision, recall, accuracy, loss = classifier.evaluate(test_generator)
print("Precision: ", precision)
print("Recall: ", recall)
print("Accuracy: ", accuracy)

341/341 ━━━━━━━━━━━━━━━━━━━━ 38s 112ms/step - Precision: 0.8241 - Recall: 0.8778 - accuracy: 0.8477 - loss: 0.5329
Precision:  0.5414142608642578
Recall:  0.8268733620643616
Accuracy:  0.8867540955543518


In [15]:
classifier.save("deepfake_autoencoder.keras")

In [2]:
model = load_model("deepfake_autoencoder.keras")

In [21]:
from sklearn2pmml import sklearn2pmml
from keras2sklearn import KerasClassifier

estimator = KerasClassifier(model=model)

sklearn2pmml(model, "deepfake_autoencoder.pmml", with_repr=True)

ModuleNotFoundError: No module named 'keras2sklearn'

In [9]:
def preprocess_image(image_path, image_size):
    image = cv2.imread(image_path)
    image = cv2.resize(image, image_size)
    image = image.astype(np.float32) / 255.0
    return np.expand_dims(image, axis=0)

def predict_image(model, image_path, image_size):
    input_image = preprocess_image(image_path, image_size)
    prediction = model.predict(input_image)
    if prediction[0][0] >= 0.9:
        return "Real", prediction[0][0]
    else:
        return "Deep Fake", prediction[0][0]

In [11]:
predict_image(model, "WhatsApp Image 2024-04-13 at 08.59.03_e36450ad.jpg", (256, 256))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step


('Real', 0.95745814)